In [0]:
schema_use = spark.sql("SELECT REPLACE(SPLIT(current_user(), '@')[0], '.', '_')").collect()[0][0]
schema_use

In [0]:
source_volume = "/Volumes/fhir_workshop/synthea/synthetic_files_raw/output/fhir/"
target_volume = f"/Volumes/fhir_workshop/{schema_use}/landing/"
target_volume

In [0]:
import shutil
import glob
import os

def copy_files(source_volume, target_volume, file_pattern):
  # Check if the source and target volumes end with a slash
  if not source_volume.endswith('/'):
    source_volume += '/'

  if not target_volume.endswith('/'):
    target_volume += '/'

  # Use glob to locate files based on the file pattern
  if file_pattern is None:
    file_pattern = '*'

  files = glob.glob(os.path.join(source_volume, file_pattern))

  # Copy each file to the destination directory
  for file in files:
      shutil.copy2(file, target_volume)

  return f"Copied {str(len(files))} files."

In [0]:
# file_patterns = ["Aa*.json", "Ab*.json", "Ad*.json", "Af*.json", "Ag*.json", "Ah*.json", "Ai*.json", "Aj*.json", "Ak*.json"]
file_patterns = ["*"]

In [0]:
for file_pattern in file_patterns:
    print(copy_files(source_volume, target_volume, file_pattern))

In [0]:
# files_to_remove = dbutils.fs.ls(target_volume)
# for file in files_to_remove:
#     if file.name.startswith("Al") and file.name.endswith(".json"):
#         dbutils.fs.rm(file.path)